In [1]:
import pandas as pd
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
import keras_tuner as kt

import math

import os


2023-07-30 20:44:55.447734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:


#IN THE FOOLOWING CLASS WE DEFINE THE SEARCH SPACE
#we tune in the following order:
#the number of layers
#the number of units
#the optimizer
#the activation function
#dropout - dropout rate
#the loss function
#the learning rate
#batch size (in fit method)
#********
#we create a keras.Model by subclassing HyperModel class and
#using the hp argument to define the hyperparameters

#HyperModel instance to the Tuner

#No batch Normalization

class Dense_model_1(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        
        self._li_activ_fcts=val_li_activ_fcts_model
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a Fully connected model.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
        new_feat= layers.Flatten()(inputs)

        nb_lay=hp.Int('num_layers',
                              min_value=self._min_nb_lay, 
                              max_value=self._max_nb_lay,
                              step=self._step_nb_layers)
        
        
        
        for i in range(nb_lay):
            
            
            new_feat=layers.Dense(\
                units=hp.Int(name="units_lay_"+str(i),\
                             min_value=self._min_nb_units,\
                             max_value=self._max_nb_units,\
                             step=self._step_nb_units),\
                activation=\
                hp.Choice(name = 'activation_'+str(i),\
                          values = self._li_activ_fcts,\
                          ordered = False))(new_feat)
            
            
        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        new_feat=layers.Dense(\
                         units=self._nb_last_output_classes,
                         )(new_feat)
        
        #we create the keras model
        model = tensorflow.keras.Model(inputs=inputs, outputs=new_feat)
        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)
        
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)
                        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)
        
        
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)


        def fit(self, hp, model, *args, **kwargs):
            return model.fit(
                *args,
               batch_size=hp.Choice("batch_size", [32, 1024]),
                **kwargs)
        
        return model
            
                         
                         
                         

In [3]:

#LayerNormalization
class Dense_model_2(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None, 
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        
    
        self._li_activ_fcts_model=val_li_activ_fcts_model
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        
       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a Fully connected model.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
        new_feat= layers.Flatten()(inputs)

        nb_lay=\
        hp.Int('num_layers',min_value=self._min_nb_lay,\
               max_value=self._max_nb_lay,step=self._step_nb_layers)

        
        
        
        for i in range(nb_lay):

            if hp.Boolean("layernormalization_"+str(i)):
                
                

                #since normalization will center the output to zero
                #the  bias vector is not neded
                #if relu is selected as activation function
                #according to some bibliography 
                #doing normalization before the activation 
                #maximizes the utilization of the relu.
                #but this ordering is not very critical
                new_feat=layers.Dense(\
                    units=\
                    hp.Int(name="units_lay_"+str(i),\
                           min_value=self._min_nb_units,\
                           max_value=self._max_nb_units,\
                           step=self._step_nb_units),\
                use_bias=False)(new_feat)

                #print("drop_rate",drop_rate)
                new_feat=layers.LayerNormalization()(new_feat)

                act_fct=\
                hp.Choice(name='activation_'+str(i),\
                          values=self._li_activ_fcts_model,\
                          ordered=False)

                new_feat=layers.Activation(act_fct)(new_feat)
                
            else:

                new_feat=layers.Dense(\
                    units=\
                    hp.Int(name="units_lay_"+str(i),\
                           min_value=self._min_nb_units,\
                           max_value=self._max_nb_units,\
                           step=self._step_nb_units),\
                    activation=\
                    hp.Choice(name='activation_'+str(i),\
                              values=self._li_activ_fcts_model,\
                              ordered=False))(new_feat)


            
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        new_feat=layers.Dense(units=self._nb_last_output_classes)(new_feat)
        #we create the keras model
        model=tensorflow.keras.Model(inputs=inputs, outputs=new_feat)

        #in order to compile the model we need to define optimizers
        optimizer=\
        hp.Choice(name="optimizer", values=self._li_optimizers)

        loss_hp=\
        hp.Choice(name = 'loss',values = self._loss,ordered = False)

        model.compile(\
            optimizer=optimizer,loss=loss_hp,metrics=self._metrics)

        learning_rate=\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer,\
                 sampling="log")

        
        K.set_value(model.optimizer.learning_rate, learning_rate)

        return model

                                  
            
            
            
                         
                         
                         

In [4]:

#Dropout
class Dense_model_3(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None, 
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        self._min_value_dropout_rate=val_min_value_dropout_rate_model
        
        self._max_value_dropout_rate=val_max_value_dropout_rate_model 

        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        
        self._step_dropout_rate=val_step_dropout_rate_model

        
        self._li_activ_fcts_model=val_li_activ_fcts_model
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a Fully connected model.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
        new_feat= layers.Flatten()(inputs)

        #the drop out rate if a dropout is selected 
        drop_rate = hp.Float("dropout_rate",\
        min_value=self._min_value_dropout_rate,\
        max_value=self._max_value_dropout_rate,\
        step=self._step_dropout_rate)

        nb_lay=\
        hp.Int('num_layers',min_value=self._min_nb_lay,max_value=self._max_nb_lay,step=self._step_nb_layers)
        
        
        
        
        for i in range(nb_lay):
            
            new_feat=layers.Dense(\
                    units=\
                    hp.Int(name="units_lay_"+str(i),\
                           min_value=self._min_nb_units,\
                           max_value=self._max_nb_units,\
                           step=self._step_nb_units),\
                    activation=\
                    hp.Choice(name='activation_'+str(i),\
                              values=self._li_activ_fcts_model,\
                              ordered=False))(new_feat)

            if hp.Boolean("dropout_"+str(i)):

                #print("drop_rate",drop_rate)
                new_feat = tensorflow.keras.layers.Dropout(rate=drop_rate)(new_feat)


            
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        new_feat=layers.Dense(units=self._nb_last_output_classes)(new_feat)
        
        #we create the keras model
        model=tensorflow.keras.Model(inputs=inputs, outputs=new_feat)

        #in order to compile the model we need to define optimizers
        optimizer=\
        hp.Choice(name="optimizer", values=self._li_optimizers)

        loss_hp=\
        hp.Choice(name = 'loss',values = self._loss,ordered = False)

        model.compile(\
            optimizer=optimizer,loss=loss_hp,metrics=self._metrics)

        learning_rate=\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer,\
                 sampling="log")


        K.set_value(model.optimizer.learning_rate, learning_rate)

        return model

                                  
            
            
            
                         
                         
                         

In [5]:
#add model 3_1 using layer normal+ drop out

In [6]:
#simple convolutional 1D model

class SimpleConv1D_model_1(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        #hp = kt.HyperParameters() 
         
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a simple convolutional 1D  model .
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        

        x = layers.Conv1D(filters=8, kernel_size=24, activation="relu")(inputs)
        x = layers.MaxPooling1D(2)(x)
        x = layers.Conv1D(filters=8, kernel_size=12, activation="relu")(x)
        x = layers.MaxPooling1D(2)(x)
        x = layers.Conv1D(filters=8, kernel_size=6, activation="relu")(x)
        x = layers.GlobalAveragePooling1D()(x)

        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        outputs = layers.Dense(self._nb_last_output_classes)(x)

        model = keras.Model(inputs, outputs)
        
        #the. last layer will  have the appropriate nb of units
        #since we have a regressionproblem the nb of units will be one
       # x=layers.Dense(\
       # units=self._nb_last_output_classes)(x)
        
        ##we create the keras model
        #model = tensorflow.keras.Model(inputs=inputs, outputs=x)
        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)
        
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)
                        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)
        
        
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)
        
        return model
            

In [7]:

#LayerNormalization
class SimpleConv1D_model_2(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None, 
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        

        self._min_nb_filters_conv1d=\
        val_min_nb_filters_conv1d

        self._max_nb_filters_conv1d=\
        val_max_nb_filters_conv1d

        self._step_nb_filters_conv1d=\
        val_step_nb_filters_conv1d


        self._min_nb_kernel_size_conv1d=\
        val_min_nb_kernel_size_conv1d

        self._max_nb_kernel_size_conv1d=\
        val_max_nb_kernel_size_conv1d

        self._val_step_nb_kernel_size_conv1d=\
        val_step_nb_kernel_size_conv1d

        self._min_pool_size=val_min_pool_size

        self._max_pool_size=val_max_pool_size

        self._step_pool_size=val_step_pool_size
        
        self._nb_last_output_classes=\
        val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        self._li_activ_fcts_model=val_li_activ_fcts_model
       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a Fully connected model.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
    

        x=inputs

        nb_lay=\
        hp.Int('num_layers',min_value=self._min_nb_lay,\
               max_value=self._max_nb_lay,step=self._step_nb_layers)

        

        nb_filters=\
        hp.Int('num_filters',min_value=self._min_nb_filters_conv1d,\
               max_value=self._max_nb_filters_conv1d,\
               step=self._step_nb_filters_conv1d)


        nb_kernel_size=\
        hp.Int('kernel_size'\
               ,min_value=self._min_pool_size,\
               max_value=self._max_pool_size,\
               step=self._val_step_nb_kernel_size_conv1d)


        nb_f=nb_filters

        nb_ks=nb_kernel_size

        ps=\
        hp.Int('pool_size',\
               min_value=self._min_nb_kernel_size_conv1d,\
               max_value=self._max_nb_kernel_size_conv1d,\
               step=self._val_step_nb_kernel_size_conv1d)


        act_fct=\
        hp.Choice(name='activation',\
                  values=self._li_activ_fcts_model,\
                  ordered=False)
        
        
        for i in range(nb_lay):
            
            #normalization centers outputs to zero
            #so the bias vector is not needed
            #according to some bibliography 
            #doing normalization before the activation 
            #maximizes the utilization of the relu.
            #but this ordering is not very critical
            if hp.Boolean("layernormalization_"+str(i)):
                
               
                
                x=layers.Conv1D(\
                filters=nb_f,\
                kernel_size=nb_ks, use_bias=False)(x)
                
                x=layers.LayerNormalization()(x)
                
                x=layers.Activation(act_fct)(x)
            else:
                x=layers.Conv1D(\
                filters=nb_f,\
                kernel_size=nb_ks, activation=act_fct)(x)
                

            x = layers.MaxPooling1D(ps)(x)

            if math.ceil(nb_ks/2)>0:
                nb_ks=math.ceil(nb_ks/2)
            
        x = layers.GlobalAveragePooling1D()(x)
        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        x=layers.Dense(units=self._nb_last_output_classes)(x)
        
        #we create the keras model
        model=tensorflow.keras.Model(inputs=inputs, outputs=x)

        #in order to compile the model we need to define optimizers
        optimizer=\
        hp.Choice(name="optimizer", values=self._li_optimizers)

        loss_hp=\
        hp.Choice(name = 'loss',values = self._loss,ordered = False)

        model.compile(\
            optimizer=optimizer,loss=loss_hp,metrics=self._metrics)

        learning_rate=\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer,\
                 sampling="log")

        
        K.set_value(model.optimizer.learning_rate, learning_rate)

        return model

                                  
            
            
            
                         
                         
                         

In [8]:

#RNN
#********
#we create a keras.Model by subclassing HyperModel class and
#using the hp argument to define the hyperparameters



#with BatchNormalization 

class RNN_model_1(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a simple RNN  model with LSTM layers.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))

        #the output will be (None, sequence length)
        x=layers.LSTM(14,return_sequences=False,activation="tanh")(inputs)
        
        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        x=layers.Dense(\
        units=self._nb_last_output_classes)(x)
        
        #we create the keras model
        model = tensorflow.keras.Model(inputs=inputs, outputs=x)
        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)
        
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)
                        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)
        
    
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)
        
        return model
            
                         
                         
                         

In [9]:

#RNN
#********
#we create a keras.Model by subclassing HyperModel class and
#using the hp argument to define the hyperparameters

#HyperModel instance to the Tuner


#with BatchNormalization 

class RNN_model_2(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        self._li_activ_fcts_model=val_li_activ_fcts_model
       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a simple RNN  model with LSTM layers.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs =\
        keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))

        act_fct=\
        hp.Choice(name='activation',\
                  values = self._li_activ_fcts_model,\
                  ordered = False)
        
        x=layers.GRU(32, recurrent_dropout=0.25, return_sequences=True,\
                     activation=act_fct)(inputs)
        x=layers.GRU(32, recurrent_dropout=0.25, return_sequences=False,\
                     activation=act_fct)(x)
        x=layers.Dropout(0.5)(x)

        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        x=layers.Dense(\
        units=self._nb_last_output_classes)(x)

        
        #we create the keras model
        model = tensorflow.keras.Model(inputs=inputs, outputs=x)

        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)

       
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)

        
        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)

        
        
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)
        
        return model
            
                         
                         
                         

In [10]:


#RNN
#********
#we create a keras.Model by subclassing HyperModel class and
#using the hp argument to define the hyperparameters



#with BatchNormalization 

class RNN_model_3(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        self._min_value_dropout_rate=val_min_value_dropout_rate_model
        
        self._max_value_dropout_rate=val_max_value_dropout_rate_model 
        
        self._min_value_recurrent_dropout_rate_model=\
        val_min_value_recurrent_dropout_rate_model

        self._max_value_recurrent_dropout_rate_model=\
        val_max_value_recurrent_dropout_rate_model
        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        
        self._step_dropout_rate=val_step_dropout_rate_model

        self._step_recurrent_dropout_rate_model=\
        val_step_recurrent_dropout_rate_model
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        self._li_activ_fcts_model=val_li_activ_fcts_model
       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a simple RNN  model with LSTM layers.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
        
        x=inputs
      

        a=hp.Int('num_layers',\
        min_value=self._min_nb_lay,\
        max_value=self._max_nb_lay,\
        step=self._step_nb_layers)
        
        #if a>1:
        #val_return_sequences=True
        #else:
        val_return_sequences=False

        
        drop_rate = hp.Float("dropout_rate",
                             min_value=self._min_value_dropout_rate,
                             max_value=self._max_value_dropout_rate,
                             step=self._step_dropout_rate) 

        va_recurrent_dropout=\
        hp.Float('rec_droput_lay',\
        min_value=self._min_value_recurrent_dropout_rate_model,\
        max_value=self._max_value_recurrent_dropout_rate_model,\
        step=self._step_recurrent_dropout_rate_model)

        #the activation function is selected
        act_fct=\
        hp.Choice(name='activation',\
                    values = self._li_activ_fcts_model,\
                    ordered = False)
        
        for i in range(a):

            #we ask the intermediate layers to return
            #the full sequence of outputs
            if i<int(a-1):
                val_return_sequences=True
            else:
                val_return_sequences=False
            
            
            #Each of the hyperparameters is uniquely identified 
            #by its name (the first argument). 
            #To tune the number of units in different Dense layers 
            #separately as different hyperparameters, 
            #we give them different names as f"units_{i}".
            
    

            v_recurrent_dropout=va_recurrent_dropout

            #the normalization wil center outputs to zero
            #so bias vector is not needed
            #if Layernormalization                     
            if hp.Boolean("layernorm_"+str(i)):

                x=layers.LSTM(\
                         units=hp.Int(name="units_lay_"+str(i), 
                         min_value=self._min_nb_units, 
                         max_value=self._max_nb_units, 
                         step=self._step_nb_units,
                         ),return_sequences=val_return_sequences,\
                         recurrent_dropout=v_recurrent_dropout,\
                         use_bias=False)(x)

                x=layers.LayerNormalization()(x)

                x=layers.Activation(act_fct)(x)
            else:
                x=layers.LSTM(\
                         units=hp.Int(name="units_lay_"+str(i), 
                         min_value=self._min_nb_units, 
                         max_value=self._max_nb_units, 
                         step=self._step_nb_units,
                         ),return_sequences=val_return_sequences,\
                         recurrent_dropout=v_recurrent_dropout,\
                         activation=act_fct)(x)

                
        

        
        #to regularize the dense layer we examine for a dropout layer after the
        #LSTM
        
        #if hp.Boolean("dropout_"+str(i)):
            
                
            #print("drop_rate",drop_rate)
        x = tensorflow.keras.layers.Dropout(rate=drop_rate)(x)
        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        x=layers.Dense(\
        units=self._nb_last_output_classes)(x)
        
        #we create the keras model
        model = tensorflow.keras.Model(inputs=inputs, outputs=x)
        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)
        
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)
                        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)
        
        
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)
        
        return model
            
                         
                         
                         

In [11]:
#un modele 

In [12]:


#RNN
#********
#we create a keras.Model by subclassing HyperModel class and
#using the hp argument to define the hyperparameters



#with BatchNormalization 

class RNN_model_4(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        self._min_value_dropout_rate=val_min_value_dropout_rate_model
        
        self._max_value_dropout_rate=val_max_value_dropout_rate_model 
        
        self._min_value_recurrent_dropout_rate_model=\
        val_min_value_recurrent_dropout_rate_model

        self._max_value_recurrent_dropout_rate_model=\
        val_max_value_recurrent_dropout_rate_model
        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        
        self._step_dropout_rate=val_step_dropout_rate_model

        self._step_recurrent_dropout_rate_model=\
        val_step_recurrent_dropout_rate_model
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        self._li_activ_fcts_model=val_li_activ_fcts_model
       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a simple RNN  model with LSTM layers.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
        
        x=inputs
        
        a=hp.Int('num_layers',\
        min_value=self._min_nb_lay,\
        max_value=self._max_nb_lay,\
        step=self._step_nb_layers)
        
        #if a>1:
        #val_return_sequences=True
        #else:
        val_return_sequences=False

        
        drop_rate = hp.Float("dropout_rate",
                             min_value=self._min_value_dropout_rate,
                             max_value=self._max_value_dropout_rate,
                             step=self._step_dropout_rate) 

        va_recurrent_dropout=\
        hp.Float('rec_droput_lay',\
        min_value=self._min_value_recurrent_dropout_rate_model,\
        max_value=self._max_value_recurrent_dropout_rate_model,\
        step=self._step_recurrent_dropout_rate_model)

        #the activation function is selected
        act_fct=\
        hp.Choice(name='activation',\
                  values = self._li_activ_fcts_model,\
                  ordered = False)
        
        for i in range(a):
            
            if i<int(a-1):
                val_return_sequences=True
            else:
                val_return_sequences=False
            
            
            #Each of the hyperparameters is uniquely identified 
            #by its name (the first argument). 
            #To tune the number of units in different Dense layers 
            #separately as different hyperparameters, 
            #we give them different names as f"units_{i}".
            
            
            #if hp.Boolean("recurrent_dropout_"+str(i)):
                
            #    v_recurrent_dropout=va_recurrent_dropout
            #else:
            #    v_recurrent_dropout=0
                                  
            #new_feat=layers.Activation(hp.Choice(name = 'activation_'+str(i),
            #                   values = self._li_activ_fcts,
            #                   ordered = False))(new_feat)
                
            #the activation function is selected
            #act_fct=\
            #hp.Choice(name='activation_'+str(i),\
            #          values = self._li_activ_fcts_model,\
            #          ordered = False)

            v_recurrent_dropout=va_recurrent_dropout

            x=layers.Bidirectional(layers.LSTM(\
                         units=hp.Int(name="units_lay_"+str(i), 
                         min_value=self._min_nb_units, 
                         max_value=self._max_nb_units, 
                         step=self._step_nb_units,
                         ),return_sequences=val_return_sequences,\
                         recurrent_dropout=v_recurrent_dropout,\
                         activation=act_fct))(x)
            
            #if Layernormalization                     
            #if hp.Boolean("layernorm_"+str(i)):

            #    x=layers.LayerNormalization()(x)

                
        

       
        #to regularize the dense layer we examine for a dropout layer after the
        #LSTM
        
        #if hp.Boolean("dropout"):
            
                
            #print("drop_rate",drop_rate)
        x = tensorflow.keras.layers.Dropout(rate=drop_rate)(x)
        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        x=layers.Dense(\
        units=self._nb_last_output_classes)(x)
        
        #we create the keras model
        model = tensorflow.keras.Model(inputs=inputs, outputs=x)
        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)
        
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)
                        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)
        
        
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)
        
        return model
            
                         
                         
                         

In [13]:


#RNN
#********
#we create a keras.Model by subclassing HyperModel class and
#using the hp argument to define the hyperparameters



#with BatchNormalization 

class RNN_model_5(kt.HyperModel):
    
    def __init__(self,
    val_s_length_train_model,
    val_nb_initial_input_features_model,
    val_min_nb_lay_model=None,
    val_max_nb_lay_model=None,
    val_min_nb_units_model=None,
    val_max_nb_units_model=None,
    val_min_value_dropout_rate_model=None,
    val_max_value_dropout_rate_model=None,
    val_min_value_recurrent_dropout_rate_model=None,
    val_max_value_recurrent_dropout_rate_model=None,
    val_min_nb_filters_conv1d=None,
    val_max_nb_filters_conv1d=None,
    val_min_nb_kernel_size_conv1d=None,
    val_max_nb_kernel_size_conv1d=None,
    val_step_nb_layers_model=None,
    val_step_nb_units_model=None,
    val_step_dropout_rate_model=None,
    val_step_recurrent_dropout_rate_model=None,
    val_step_nb_filters_conv1d=None,
    val_step_nb_kernel_size_conv1d=None,
    val_min_pool_size=None,
    val_max_pool_size=None,
    val_step_pool_size=None,
    val_li_activ_fcts_model=None,
    val_nb_last_output_classes_model=None,
    val_li_optimizers_model=None,
    val_min_val_learning_rate_optimizer=None,
    val_max_val_learning_rate_optimizer=None,
    var_loss_fct_model=None,
    #tensorflow.keras.metrics.MeanSquaredLogarithmicError(
    #name="mean_squared_logarithmic_error")],
    var_metrics_model=None
    ):
        
        self._s_length_train=val_s_length_train_model
        
        self._nb_initial_input_features=val_nb_initial_input_features_model
        
        self._min_nb_lay=val_min_nb_lay_model
        
        self._max_nb_lay=val_max_nb_lay_model
        
        self._min_nb_units=val_min_nb_units_model
        
        self._max_nb_units=val_max_nb_units_model
        
        self._min_value_dropout_rate=val_min_value_dropout_rate_model
        
        self._max_value_dropout_rate=val_max_value_dropout_rate_model 
        
        self._min_value_recurrent_dropout_rate_model=\
        val_min_value_recurrent_dropout_rate_model

        self._max_value_recurrent_dropout_rate_model=\
        val_max_value_recurrent_dropout_rate_model
        
        self._step_nb_layers=val_step_nb_layers_model
        
        self._step_nb_units=val_step_nb_units_model
        
        self._step_dropout_rate=val_step_dropout_rate_model

        self._step_recurrent_dropout_rate_model=\
        val_step_recurrent_dropout_rate_model
        
        self._nb_last_output_classes=val_nb_last_output_classes_model
                       
        self._li_optimizers=val_li_optimizers_model

        self._min_val_learning_rate_optimizer=\
        val_min_val_learning_rate_optimizer

        self._max_val_learning_rate_optimizer=\
        val_max_val_learning_rate_optimizer
        
        self._loss=var_loss_fct_model
        
        self._metrics=var_metrics_model

        self._li_activ_fcts_model=val_li_activ_fcts_model
       

        #hp = kt.HyperParameters() 
        
        
    
    #method returning a compiled model 
    def build(self,hp):
        
        """
        Builds a simple RNN  model with LSTM layers.
    
        Args:
        hp: instance  of Hyperparameter class of the Keras Tuner package, 
        This object defines the search space for the hyperparameter values
        for a particular trial.
        
        Returns:
        a Keras model.
        """
        
        #we define the input layer
        inputs = keras.Input(shape=(self._s_length_train,self._nb_initial_input_features))
        
        
        x=inputs
        

        a=hp.Int('num_layers',\
        min_value=self._min_nb_lay,\
        max_value=self._max_nb_lay,\
        step=self._step_nb_layers)
        
        #if a>1:
        #val_return_sequences=True
        #else:
        val_return_sequences=False

        
        drop_rate = hp.Float("dropout_rate",
                             min_value=self._min_value_dropout_rate,
                             max_value=self._max_value_dropout_rate,
                             step=self._step_dropout_rate) 

        va_recurrent_dropout=\
        hp.Float('rec_droput_lay',\
        min_value=self._min_value_recurrent_dropout_rate_model,\
        max_value=self._max_value_recurrent_dropout_rate_model,\
        step=self._step_recurrent_dropout_rate_model)


        #the activation function is selected
        act_fct=\
        hp.Choice(name='activation',\
                  values = self._li_activ_fcts_model,\
                  ordered = False)
        
        for i in range(a):
            
            if i<int(a-1):
                val_return_sequences=True
            else:
                val_return_sequences=False
            
            
            #Each of the hyperparameters is uniquely identified 
            #by its name (the first argument). 
            #To tune the number of units in different Dense layers 
            #separately as different hyperparameters, 
            #we give them different names as f"units_{i}".
            
            
            v_recurrent_dropout=va_recurrent_dropout

            
            #if Layernormalization 
            #since normalization will center the output at zero
            #no need of bias vector
            if hp.Boolean("layernorm_"+str(i)):

                x=layers.Bidirectional(layers.GRU(\
                    units=hp.Int(name="units_lay_"+str(i),\
                                 min_value=self._min_nb_units,\
                                 max_value=self._max_nb_units,\
                                 step=self._step_nb_units),\
                    return_sequences=val_return_sequences,\
                    recurrent_dropout=v_recurrent_dropout,\
                    use_bias=False))(x)
                
                x=layers.LayerNormalization()(x)

                x=layers.Activation(act_fct)(x)
            else:

                x=layers.Bidirectional(layers.GRU(\
                    units=hp.Int(name="units_lay_"+str(i),\
                                 min_value=self._min_nb_units,\
                                 max_value=self._max_nb_units,\
                                 step=self._step_nb_units),\
                    return_sequences=val_return_sequences,\
                    recurrent_dropout=v_recurrent_dropout,\
                    activation=act_fct))(x)

                
        

        
        #to regularize the dense layer we examine for a dropout layer after the
        #LSTM
        
        #if hp.Boolean("dropout"):
            
                
            #print("drop_rate",drop_rate)
        x = tensorflow.keras.layers.Dropout(rate=drop_rate)(x)
        
        #since we have a regression problem 
        #the nb of units of the last layer will be
        #the number of timesteps to predict
        #no activation of thelast layer 
        #so as to not constrain the output.
        #If the last layer is linear the model wil learn
        #to predict values of any range
        x=layers.Dense(\
        units=self._nb_last_output_classes)(x)
        
        #we create the keras model
        model = tensorflow.keras.Model(inputs=inputs, outputs=x)
        
        #in order to compile the model we need to define optimizers
        optimizer= hp.Choice(name="optimizer", values=self._li_optimizers)
        
        loss_hp=hp.Choice(name = 'loss',
        values = self._loss,ordered = False)
                        
        model.compile(
            optimizer=optimizer,
            loss=loss_hp,
            #loss=self._loss,
            metrics=self._metrics)
        
        
        learning_rate =\
        hp.Float("lr", min_value=self._min_val_learning_rate_optimizer,\
                 max_value=self._max_val_learning_rate_optimizer, sampling="log")
        
        
        K.set_value(model.optimizer.learning_rate, learning_rate)
        
        return model
            
                         
                         
                         

In [14]:
#di_models=dict,  AFAIRE BIEN
#key = number, value=name class creating the hypermodel


di_hypermodels={1:Dense_model_1,\
               2:Dense_model_2,\
               3:Dense_model_3,\
               4:SimpleConv1D_model_1,\
               5:SimpleConv1D_model_2,\
               6:RNN_model_1,\
               7:RNN_model_2,\
               8:RNN_model_3,\
               9:RNN_model_4,\
               10:RNN_model_5}

In [15]:


class BayesianOptimizationTuner_j(kt.tuners.BayesianOptimization):
    
    def run_trial(self, trial, *args, **kwargs):
        
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', \
                                                        32,2048,32)
        
        return super(BayesianOptimizationTuner_j, self).run_trial(trial, *args, **kwargs)

In [16]:
di_tuners={1:kt.tuners.BayesianOptimization,2:BayesianOptimizationTuner_j,
          3:kt.GridSearch}

In [17]:
#list with the keys of the tuners which optimize the batch size
#this list  must be updated whenever a new tuner tuning the batch size is considered
val_li_keys_tuners_optimizing_batch_size=[2]


In [18]:
#fct which returns t the list with the tuners tuning
# the batch size
def fct_li_keys_tuners_tuning_batch_size():
    return val_li_keys_tuners_optimizing_batch_size
    
